# extracting accessibility related patterns from ravelry

In [1]:
%%
## bootstrap our search for accessible ravelry patterns

there are several accessibility categories that need to be searched.
they are captured in the `searches` mapping where the key is the name/index
for the search and the value defines the query parameters.

    __import__("dotenv").load_dotenv()
    auth = (os.environ["RAVELRY_USERNAME"], os.environ["RAVELRY_PASSWORD"])
    import httpx    
    searches: dict =\
```toml
adaptive.pa = "adaptive"
"medical device access".pa = "medical-device-access"
"medical device support".pa = "medical-device-accessory"
"mobility aid support".pa = "mobility-aid-accesory"
other.pa = "other-add-accessibility"
"therapy aid/toy".pa = "therapy-aid"
medical.pc = "medical"
```

the `seed_urls` provide the `first_pages` returned from  our `searches`

    seed_urls = ("https://api.ravelry.com/patterns/search.json?" + Series(searches).apply(urllib.parse.urlencode))
    first_pages = seed_urls.apply(httpx.get, auth=auth)

{{first_pages.to_frame("responses").T._repr_html_()}}

bootstrap our search for accessible ravelry patterns 
 there are several accessibility categories that need to be searched.
they are captured in the searches mapping where the key is the name/index
for the search and the value defines the query parameters. 
 __import__("dotenv").load_dotenv()
auth = (os.environ["RAVELRY_USERNAME"], os.environ["RAVELRY_PASSWORD"])
import httpx 
searches: dict =\
 
 adaptive . pa = "adaptive" 
 "medical device access" . pa = "medical-device-access" 
 "medical device support" . pa = "medical-device-accessory" 
 "mobility aid support" . pa = "mobility-aid-accesory" 
 other . pa = "other-add-accessibility" 
 "therapy aid/toy" . pa = "therapy-aid" 
 medical . pc = "medical" 
 
 
 the seed_urls provide the first_pages returned from our searches 
 seed_urls = ("https://api.ravelry.com/patterns/search.json?" + Series(searches).apply(urllib.parse.urlencode))
first_pages = seed_urls.apply(httpx.get, auth=auth)
 
 

 
 
 
 
 adaptive 
 medical device access 
 medical device support 
 mobility aid support 
 other 
 therapy aid/toy 
 medical 
 
 
 
 
 responses 
 <Response [200 OK]> 
 <Response [200 OK]> 
 <Response [200 OK]> 
 <Response [500 Internal Server Error]> 
 <Response [500 Internal Server Error]> 
 <Response [200 OK]> 
 <Response [200 OK]>

In [2]:
%%
### handling bad `searches`

not all of the queries are dialed in. the `searches` that don't return a 200 status code are a failure.

    status_codes = first_pages.attrgetter("status_code")
{{status_codes.rename_axis("search", axis=0).to_frame("status code").T._repr_html_()}}

{{first_pages.index[status_codes.ne(200)].map("<samp>{}</samp>".format) | join(", ")}} are not working properly yet.

handling bad searches 
 not all of the queries are dialed in. the searches that don't return a 200 status code are a failure. 
 status_codes = first_pages.attrgetter("status_code")
 
 

 
 
 
 search 
 adaptive 
 medical device access 
 medical device support 
 mobility aid support 
 other 
 therapy aid/toy 
 medical 
 
 
 
 
 status code 
 200 
 200 
 200 
 500 
 500 
 200 
 200 
 
 
 
 
 mobility aid support , other are not working properly yet.

In [3]:
%%
## finding the missing paginated search information

from the `first_pages` were learn about possible results we missed in the `first_pattern.paginator` attribute in the payload.

    first_patterns = first_pages[status_codes.eq(200)].methodcaller("json").series()
    paginated = first_patterns.paginator.series()
    paginated = paginated[paginated.page_count.gt(1)].drop(columns="page")
    paginated = paginated.join(paginated.page_count.add(1).apply(compose(list, partial(range, 2))).explode().rename("page"))
from `paginated`, we can determine the other `searches` we need to make to complete our list of patterns;
the responses don't have much pattern information and we dig deeper in a few more calls.

    other_urls = seed_urls[paginated.index] + "&page=" + paginated.page.astype(str)
we leave with the `other_pages` that can be combined with the `first_pages` to provide
all of the `responses`.

    other_pages = other_urls.apply(httpx.get, auth=auth)
    responses = pandas.concat([first_pages, other_pages]).to_frame("response")

finding the missing paginated search information 
 from the first_pages were learn about possible results we missed in the first_pattern.paginator attribute in the payload. 
 first_patterns = first_pages[status_codes.eq(200)].methodcaller("json").series()
paginated = first_patterns.paginator.series()
paginated = paginated[paginated.page_count.gt(1)].drop(columns="page")
paginated = paginated.join(paginated.page_count.add(1).apply(compose(list, partial(range, 2))).explode().rename("page"))
 
 from paginated , we can determine the other searches we need to make to complete our list of patterns;
the responses don't have much pattern information and we dig deeper in a few more calls. 
 other_urls = seed_urls[paginated.index] + "&page=" + paginated.page.astype(str)
 
 we leave with the other_pages that can be combined with the first_pages to provide
all of the responses . 
 other_pages = other_urls.apply(httpx.get, auth=auth)
responses = pandas.concat([first_pages, other_pages]).to_frame("response")

In [4]:
%%
### extract all of the `pattern_ids` we have information about

extract the good `responses` and parse their results into a table of `pattern_ids`

    responses = responses.assign(
        url=responses.response.attrgetter("url"),
        status_code=responses.response.attrgetter("status_code"),
    )
    responses = responses[responses.status_code.eq(200)]
    responses = responses.assign(**responses.response.methodcaller("json").series())

    pattern_ids = responses.patterns.explode().series().reset_index().drop_duplicates("id").set_index("id")

extract all of the pattern_ids we have information about 
 extract the good responses and parse their results into a table of pattern_ids 
 responses = responses.assign(
 url=responses.response.attrgetter("url"),
 status_code=responses.response.attrgetter("status_code"),
)
responses = responses[responses.status_code.eq(200)]
responses = responses.assign(**responses.response.methodcaller("json").series())

pattern_ids = responses.patterns.explode().series().reset_index().drop_duplicates("id").set_index("id")

In [5]:
%%
## extract information about each actual pattern

we can use the `patterns.json` endpoint to each information about each pattern.
there doesn't seem to be an upper limit. we are bound by a time out so the number 
we partition on can take too long and fail to respond.

    pattern_urls = pipe(
        len(pattern_ids),
        range,
        partition(50),
        map(compose(
            "https://api.ravelry.com/patterns.json?ids={}".format, "+".join, map(compose(str, pattern_ids.index.__getitem__))
        )),list, Series)

    limit = 3 # limit the queries when reporting
    pattern_pages = pattern_urls.iloc[:limit].apply(httpx.get, auth=auth)

extract information about each actual pattern 
 we can use the patterns.json endpoint to each information about each pattern.
there doesn't seem to be an upper limit. we are bound by a time out so the number
we partition on can take too long and fail to respond. 
 pattern_urls = pipe(
 len(pattern_ids),
 range,
 partition(50),
 map(compose(
 "https://api.ravelry.com/patterns.json?ids={}".format, "+".join, map(compose(str, pattern_ids.index.__getitem__))
 )),list, Series)

limit = 3 # limit the queries when reporting
pattern_pages = pattern_urls.iloc[:limit].apply(httpx.get, auth=auth)

In [6]:
%%
## all of the pattern information

    patterns = pattern_pages.methodcaller("json").series().patterns.apply(compose(list, dict.values)).explode().series()
    

{{patterns.head().T.fillna("")._repr_html_()}}

all of the pattern information 
 patterns = pattern_pages.methodcaller("json").series().patterns.apply(compose(list, dict.values)).explode().series()
 
 

 
 
 
 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 comments_count 
 6 
 23 
 23 
 13 
 32 
 
 
 created_at 
 2008/03/28 14:40:29 -0400 
 2008/05/17 10:10:10 -0400 
 2009/01/14 12:30:32 -0500 
 2009/04/04 14:42:17 -0400 
 2011/03/20 18:29:09 -0400 
 
 
 currency 
 
 
 USD 
 USD 
 
 
 
 difficulty_average 
 2.5 
 2.788991 
 1.801282 
 1.811881 
 2.666667 
 
 
 difficulty_count 
 14.0 
 109.0 
 156.0 
 101.0 
 72.0 
 
 
 downloadable 
 True 
 True 
 True 
 False 
 True 
 
 
 favorites_count 
 1055 
 1978 
 2767 
 1937 
 2640 
 
 
 free 
 True 
 True 
 True 
 False 
 True 
 
 
 gauge 
 8.0 
 24.0 
 14.0 
 8.0 
 4.0 
 
 
 gauge_divisor 
 1.0 
 4.0 
 4.0 
 4.0 
 1.0 
 
 
 gauge_pattern 
 stocking stitch 
 Stockinette stitch 
 double crochet (US)/treble (UK) 
 garter stitch 
 stockinette 
 
 
 generally_available 
 2008/03/28 14:40:29 -0400 
 2010/10/01 00:00:00 -0400 
 2008/12/01 00:00:00 -0500 
 2008/09/01 00:00:00 -0400 
 2011/03/20 18:29:09 -0400 
 
 
 has_uk_terminology 
 
 
 
 
 
 
 
 has_us_terminology 
 
 
 True 
 
 
 
 
 id 
 58010 
 65601 
 103495 
 117160 
 235375 
 
 
 name 
 Comfy Soled Socks 
 Phil's Knee Warmer 
 Wheelchair Wrap 
 Cloud On Her Shoulders 
 Old Shale Face Warmer 
 
 
 pdf_url 
 
 
 
 
 
 
 
 permalink 
 comfy-soled-socks 
 phils-knee-warmer 
 wheelchair-wrap 
 cloud-on-her-shoulders 
 old-shale-face-warmer 
 
 
 price 
 
 
 
 
 
 
 
 projects_count 
 48 
 363 
 487 
 304 
 273 
 
 
 published 
 2008/03/01 
 2010/10/01 
 2008/12/01 
 2008/09/01 
 2011/03/01 
 
 
 queued_projects_count 
 177 
 304 
 395 
 282 
 414 
 
 
 rating_average 
 4.4 
 4.264706 
 4.587838 
 4.464646 
 3.928571 
 
 
 rating_count 
 15.0 
 102.0 
 148.0 
 99.0 
 70.0 
 
 
 row_gauge 
 
 
 8.0 
 10.0 
 
 
 
 updated_at 
 2019/08/30 13:27:23 -0400 
 2024/08/22 04:35:27 -0400 
 2022/09/10 06:58:54 -0400 
 2024/11/14 09:27:11 -0500 
 2019/10/01 13:19:15 -0400 
 
 
 url 
 http://dogloversyarn.blogspot.com/2008/03/mult... 
 
 http://needleworkersroom.blogspot.com/2008/12/... 
 
 
 
 
 yardage 
 
 120.0 
 
 220.0 
 60.0 
 
 
 yardage_max 
 
 150.0 
 
 440.0 
 70.0 
 
 
 yarn_list_type 
 1.0 
 2.0 
 1.0 
 1.0 
 1.0 
 
 
 personal_attributes 
 
 
 
 
 
 
 
 sizes_available 
 Women's Small, Medium, and Large 
 medium, large 
 One size 
 one size fits to 4xl, easily sized up by addin... 
 
 
 
 product_id 
 8841.0 
 47836.0 
 
 
 61319.0 
 
 
 unlisted_product_ids 
 
 
 
 
 
 
 
 currency_symbol 
 
 
 $ 
 $ 
 
 
 
 ravelry_download 
 True 
 True 
 False 
 False 
 True 
 
 
 download_location 
 {'type': 'ravelry', 'free': True, 'url': 'http... 
 {'type': 'ravelry', 'free': True, 'url': 'http... 
 {'type': 'external', 'free': True, 'url': 'htt... 
 
 {'type': 'ravelry', 'free': True, 'url': 'http... 
 
 
 pdf_in_library 
 False 
 False 
 False 
 False 
 False 
 
 
 volumes_in_library 
 
 
 
 
 
 
 
 gauge_description 
 8 stitches = 1 inch in stocking stitch 
 24 stitches = 4 inches in Stockinette stitch 
 14 stitches and 8 rows = 4 inches in double cr... 
 8 stitches and 10 rows = 4 inches in garter st... 
 4 stitches = 1 inch in stockinette 
 
 
 yarn_weight_description 
 Fingering (14 wpi) 
 Worsted (9 wpi) 
 Worsted (9 wpi) 
 Bulky (7 wpi) 
 Aran (8 wpi) 
 
 
 yardage_description 
 yards 
 120 - 150 yards 
 yards 
 220 - 440 yards 
 60 - 70 yards 
 
 
 pattern_needle_sizes 
 [{'id': 19, 'us': '0', 'metric': 2.0, 'us_stee... 
 [{'id': 5, 'us': '5 ', 'metric': 3.75, 'us_ste... 
 [{'id': 8, 'us': '8.0', 'metric': 5.0, 'us_ste... 
 [{'id': 14, 'us': '15 ', 'metric': 10.0, 'us_s... 
 [{'id': 8, 'us': '8 ', 'metric': 5.0, 'us_stee... 
 
 
 notes_html 
 \n<p>A basic sock with reinforced heel flap, h... 
 \n<p>My father-in-law, after his knee replacem... 
 \n<p>Pattern calls for 12 ounces worsted weigh... 
 \n<p>In exchange for downloading this pattern,... 
 \n<p>A face warmer designed to work well with ... 
 
 
